In [1]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)
test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True
)

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [3]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [6]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [8]:
train_data.targets.size()

torch.Size([60000])

In [10]:
from torch.utils.data import DataLoader

loaders = {

    'train': DataLoader(train_data,
                        batch_size=100,
                        shuffle=True,
                        num_workers=1),
    
    'test': DataLoader(test_data,
                        batch_size=100,
                        shuffle=True,
                        num_workers=1)

}

In [11]:
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x339b95280>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x339b96300>}

In [12]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        self = self.fc2(x)

        return F.softmax(x)

In [39]:
import torch
print(torch.backends.mps.is_available())

# device = torch.device('mps' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
#device = torch.device('mps')

model = CNN().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loaders['train']):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(loaders["train"].dataset)} ({100. * batch_idx/len(loaders["train"]):.0f}%)]\t{loss.item():.6f}')

def test():
    model.eval()

    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loaders['test']:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(loaders['test'].dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy {correct}/{len(loaders["test"].dataset)} ({100. * correct / len(loaders["test"].dataset):.0f}%\n)')
    


True


In [40]:
for epoch in range(1,11):
    train(epoch)
    test()

/var/folders/8_/qqhcw4rd4m72212dbpl453gh0000gn/T/ipykernel_47958/845714647.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epoch: 1 [0/60000 (0%)]	3.912269
Train Epoch: 1 [2000/60000 (3%)]	3.838792
Train Epoch: 1 [4000/60000 (7%)]	3.830010
Train Epoch: 1 [6000/60000 (10%)]	3.742484
Train Epoch: 1 [8000/60000 (13%)]	3.594679
Train Epoch: 1 [10000/60000 (17%)]	3.654205
Train Epoch: 1 [12000/60000 (20%)]	3.568914
Train Epoch: 1 [14000/60000 (23%)]	3.564157
Train Epoch: 1 [16000/60000 (27%)]	3.602717
Train Epoch: 1 [18000/60000 (30%)]	3.548820
Train Epoch: 1 [20000/60000 (33%)]	3.600796
Train Epoch: 1 [22000/60000 (37%)]	3.516576
Train Epoch: 1 [24000/60000 (40%)]	3.586624
Train Epoch: 1 [26000/60000 (43%)]	3.598437
Train Epoch: 1 [28000/60000 (47%)]	3.531099
Train Epoch: 1 [30000/60000 (50%)]	3.535646
Train Epoch: 1 [32000/60000 (53%)]	3.554909
Train Epoch: 1 [34000/60000 (57%)]	3.564948
Train Epoch: 1 [36000/60000 (60%)]	3.481260
Train Epoch: 1 [38000/60000 (63%)]	3.600493
Train Epoch: 1 [40000/60000 (67%)]	3.425917
Train Epoch: 1 [42000/60000 (70%)]	3.579615
Train Epoch: 1 [44000/60000 (73%)]	3.536091

In [36]:
import time

device = torch.device('cpu')
start_time_cpu = time.time()

for epoch in range(1,11):
    train(epoch)
    test()

end_time_cpu = time.time() 
total_time_cpu = end_time_cpu - start_time_cpu

device = torch.device('mps') #gpu
start_time_gpu = time.time()

for epoch in range(1,11):
    train(epoch)
    test()

end_time_gpu = time.time() 
total_time_gpu = start_time_gpu - end_time_gpu






# print(f"Training on {device_name} took: {total_time:.2f} seconds")
    
# return total_time


# time_difference = cpu_time - gpu_time
# print(f"Time difference (CPU - GPU): {time_difference:.2f} seconds")


device(type='mps')